In [1]:
%pip install -q global-land-mask pyproj

Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
from global_land_mask import globe
from pyproj import Geod
import plotly.io as pio

# Геодезический объект для WGS-84
GEOD = Geod(ellps="WGS84")

In [3]:
def land_meridian_length(
        lon_deg: float,
        step: float = 0.00,
) -> tuple[float, float]:
    """
    Возвращает:
        (total_m_land_km, total_m_km)
    где
        total_m_land_km – суммарная длина сухопутных сегментов,
        total_m_km      – полная длина меридиана при заданном шаге.

    Parameters
    ----------
    lon_deg : float
        Долгота меридиана (–180 … +180 °).
    step : float, optional
        Шаг по широте, ° (по умолчанию 0.01° ≈ 1.1 км).
    """
    if not -180.0 <= lon_deg <= 180.0:
        raise ValueError("Долгота должна быть в диапазоне –180…+180°")
    if step <= 0 or step > 1:
        raise ValueError("step должен быть > 0 и ≤ 1°")

    # Равномерные широты, –90…+90 включительно (linspace гарантирует конец)
    n = int(round(180 / step)) + 1
    lats = np.linspace(-90.0, 90.0, n)

    # Маска суши (True=суша, False=вода)
    land_mask = globe.is_land(lats, np.full_like(lats, lon_deg))

    total_m = 0.0  # полная длина
    total_m_land = 0.0  # только суша

    # Идём по каждой паре соседних широт
    for i in range(len(lats) - 1):
        lat1, lat2 = lats[i], lats[i + 1]

        # Геодезическая длина маленького отрезка
        _, _, seg = GEOD.inv(lon_deg, lat1, lon_deg, lat2)
        seg = abs(seg)  # метры
        total_m += seg

        # На суше, только если обе точки сегмента – суша
        if land_mask[i] and land_mask[i + 1]:
            total_m_land += seg

    # Переводим в километры
    return total_m_land / 1_000.0, total_m / 1_000.0

In [4]:
from tqdm.auto import tqdm

meridians = list(range(-180, 180))
meridians_land_lengths = []

for meridian_ in tqdm(meridians):
    land_km, full_km = land_meridian_length(meridian_, step=0.01)
    meridians_land_lengths.append(land_km)

  0%|          | 0/360 [00:00<?, ?it/s]

In [9]:
import plotly.express as px

fig = px.line(
    x=meridians,
    y=meridians_land_lengths,
    labels={
        "x": "Долгота, °",
        "y": "Длина суши, км",
    },
)
fig.show()

pio.write_image(
    fig, 
    "longitude_land_len_line.png", 
    "png", 
    scale=2.0, 
    width=1280, 
    height=480
)

In [11]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np

df = pd.DataFrame({"lon": meridians, "land_km": meridians_land_lengths})
df = df.sort_values("lon")
df["lat"] = 0  # Кладём всё дело на экватор

marker_size = df["land_km"] / max(meridians_land_lengths) * 30

fig_geo = go.Figure()

# Используем colorscale в маркерах вместо линии
fig_geo.add_trace(go.Scattergeo(
    lon=df["lon"],
    lat=df["lat"],
    mode="lines+markers",
    marker=dict(
        size=marker_size,  # Теперь размер точек пропорционален длине суши
        color=df["land_km"],
        colorscale="Viridis",  # Включил обратно для лучшей визуализации
        colorbar=dict(title="Длина суши (км)")
    ),
    line=dict(width=2, color="rgba(0,0,0,0.5)"),
    name="Длина суши по меридианам"
))

fig_geo.update_geos(
    projection_type="natural earth",
    showcoastlines=True,
    showcountries=True,
    showland=True,
    landcolor="lightgray"
)

fig_geo.update_layout(
    title="Сколько суши приходится на каждый меридиан",
    height=600,
    width=1000
)

fig_geo.show()

pio.write_image(
    fig_geo, 
    "longitude_land_len_globe.png", 
    "png", 
    scale=1.0, 
    width=1280, 
    height=720
)